In [1]:
import pandas as pd
import numpy as np
import spacy

### save 10% for heldout test, use 90% for cv

In [15]:
df = pd.read_csv("./data/train.csv", index_col="ID")
split_size = int(df.shape[0]/10)

In [16]:
np.random.seed(0)
test_idx = np.random.choice(df.index, size=split_size)
rest = np.array([i for i in df.index if i not in test_idx])
val_idx = np.random.choice(rest, size=split_size)
train_idx = np.array([i for i in rest if i not in val_idx])

In [17]:
df.loc[train_idx].to_csv("./data/train_real.csv")
df.loc[val_idx].to_csv("./data/val.csv")
df.loc[test_idx].to_csv("./data/holdout_test.csv")

### Vocabulary exploration

In [11]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

In [52]:
df_train = pd.read_csv("./data/hw1_train_real.csv")
df_val = pd.read_csv("./data/hw1_val.csv")

In [51]:
en = English()
tok = Tokenizer(en.vocab)
train_vocab = set(tok(" ".join(df_train["UTTERANCE"])).vocab.strings)

In [53]:
val_vocab = set(tok(" ".join(df_val["UTTERANCE"])).vocab.strings)

In [54]:
len([i for i in val_vocab if i not in train_vocab])

90

### save all the vocabulary

In [70]:
import numpy as np

In [71]:
df1 = pd.read_csv("./data/original_data/hw1_train.csv")
df2 = pd.read_csv("./data/original_data/hw1_test.csv")

vocab1 = set(tok(" ".join(df1["UTTERANCE"])).vocab.strings)
vocab2 = set(tok(" ".join(df2["UTTERANCE"])).vocab.strings)

vocab = vocab1.union(vocab2)

In [68]:
np.save("./data/vocab.npy", np.array(sorted(list(vocab))))

### Turn labels into Manyhot representation

In [2]:
df = pd.read_csv("./data/original_data/hw1_train.csv")

In [3]:
labels = sorted(list(set(" ".join(df["CORE RELATIONS"]).split(" "))))
np.save("./data/labels.npy", labels)

In [4]:
label2idx = {text: i for i, text in enumerate(labels)}
idx2label = {i: text for i, text in enumerate(labels)}

In [9]:
def lable_mapper(labels):
    return ",".join([str(label2idx[i]) for i in labels.split(" ")])

def label2manyhot(core_relations):
    manyhot = [0] * len(labels)
    for each_label in core_relations.split(" "):
        idx = label2idx[each_label]
        manyhot[idx] = 1
    return "".join([str(i) for i in manyhot])

In [10]:
df["label"] = df["CORE RELATIONS"].apply(label2manyhot)

### rename text and label columns and save to final version

In [12]:
df["text"] = df["UTTERANCE"]
df[["ID", "text", "label"]].set_index("ID").to_csv("./data/train.csv")

In [13]:
df_test = pd.read_csv("./data/original_data/hw1_test.csv")

In [14]:
df_test["text"] = df_test["UTTERANCE"]
df[["ID", "text"]].set_index("ID").to_csv("./data/test.csv")